[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AlpacaDB/jquants-api-sample/blob/main/jquants_api_sample.ipynb)

# J-Quantsチュートリアル用データ取得

本ノートブックでは、J-Quantsチュートリアルセミナーに先立って
セミナーで必要となる株価・財務情報データをJ-Quants APIを使用して取得し、
Google Driveに保存するところまでを行います。
本ノートブックはGoogle Colaboratory上で動作確認を行っています。


## J-Quants API
J-Quants APIにアクセスするためには [J-Quants API の Web サイト](https://application.jpx-jquants.com/) から取得できるリフレッシュトークンが必要になります。
会員登録とリフレッシュトークンの取得を完了させてから次に進みます。

In [26]:
# 必要なライブラリのインストール
! pip install jquants-api-client == 0.1.2

# インポート
import getpass
from requests import HTTPError
from datetime import datetime
import time

import jquantsapi
import pandas as pd
import numpy as np
from dateutil import tz

# 必要なコンフィグの定義
# データを保存するGoogleドライブ上のディレクトリ
STORAGE_DIR_PATH = "/content/drive/MyDrive/drive_ws/marketdata"
# for local debug
STORAGE_DIR_PATH = "/tmp/marketdata"
# J-Quants API から取得するデータの期間
start_dt: datetime = datetime(2017, 1, 1, tzinfo=tz.gettz("Asia/Tokyo"))
# for debug
start_dt: datetime = datetime(2022, 7, 18, tzinfo=tz.gettz("Asia/Tokyo"))

end_dt: datetime = datetime(2022, 7, 31, tzinfo=tz.gettz("Asia/Tokyo"))

codes = []

You should consider upgrading via the '/Users/dakimura/Library/Caches/pypoetry/virtualenvs/jquants-sample-WjxObdxd-py3.9/bin/python -m pip install --upgrade pip' command.


In [3]:
# https://application.jpx-jquants.com/ から取得したJ-Quants APIのリフレッシュトークンを入力してください。
refresh_token = getpass.getpass(prompt="J-Quants Refresh Token:")
# リフレッシュトークンを使用できるか検証します。
test_cli = jquantsapi.Client(refresh_token=refresh_token)
try:
    id_token = test_cli.get_id_token()
    if len(id_token) > 0:
        print("refresh_tokenは正常です。次の手順に進んでください。")
except HTTPError:
    print("refresh_tokenを使用できません。再度値を確認してください。")

refresh_tokenは正常です。次の手順に進んでください。


データ取得期間(start_dt <= 時刻 <= end_dt) の全銘柄の価格と財務情報データを取得します。
このステップの実行には環境によって数十分ほど時間がかかることがあります。

In [36]:
cli = jquantsapi.Client(refresh_token=refresh_token)
stock_price: pd.DataFrame = cli.get_price_range(start_dt=start_dt, end_dt=end_dt)
stock_list: pd.DataFrame = stock_price["Code"].unique()
stock_fins: pd.DataFrame = cli.get_statements_range(start_dt=start_dt, end_dt=end_dt)

[株式分析チュートリアル](https://japanexchangegroup.github.io/J-Quants-Tutorial/#introduction)で用いられた株価データを[J-Quants API](https://jpx.gitbook.io/j-quants-api/api-reference)を用いて再現するためにDataFrameを変更します。

In [34]:
stock_price_output: pd.DataFrame = pd.DataFrame()
stock_price_output["Local Code"] = stock_price["Code"]
stock_price_output["Date"] = stock_price["Date"]
stock_price_output["EndOfDayQuote Open"] = stock_price["AdjustmentOpen"].replace({0.0: np.nan})
stock_price_output["EndOfDayQuote High"] = stock_price["AdjustmentHigh"].replace({0.0: np.nan})
stock_price_output["EndOfDayQuote Low"] = stock_price["AdjustmentLow"].replace({0.0: np.nan})
stock_price_output["EndOfDayQuote Close"] = stock_price["AdjustmentClose"].replace({0.0: np.nan})
stock_price_output["EndOfDayQuote ExchangeOfficialClose"] = stock_price["AdjustmentClose"].replace({0.0: np.nan})
stock_price_output["EndOfDayQuote Volume"] = stock_price["AdjustmentVolume"]
stock_price_output.set_index("Date").sort_index().groupby("Local Code").ffill()

stock_price_output

,Local Code,Date,EndOfDayQuote Open,EndOfDayQuote High,EndOfDayQuote Low,EndOfDayQuote Close,EndOfDayQuote Volume,EndOfDayQuote ExchangeOfficialClose
0,13010,2022-07-19,3660.0,3660.0,3585.0,3605.0,10900.0,3605.0
0,13010,2022-07-20,3660.0,3660.0,3610.0,3630.0,18300.0,3630.0
0,13010,2022-07-21,3585.0,3645.0,3585.0,3645.0,10300.0,3645.0
0,13010,2022-07-22,3610.0,3635.0,3605.0,3620.0,10700.0,3620.0
0,13010,2022-07-25,3615.0,3660.0,3615.0,3630.0,8100.0,3630.0
...,...,...,...,...,...,...,...,...
4193,99970,2022-07-25,829.0,831.0,816.0,826.0,151200.0,826.0
4193,99970,2022-07-26,826.0,827.0,816.0,825.0,133600.0,825.0
4194,99970,2022-07-27,819.0,822.0,811.0,811.0,136500.0,811.0
4193,99970,2022-07-28,813.0,816.0,801.0,816.0,187300.0,816.0


In [ ]:
stock_fins_output: pd.DataFrame = pd.DataFrame()
stock_fins_output["Result_FinancialStatement FiscalPeriodEnd"] = stock_fins["CurrentFiscalYearEndDate"]
#stock_fins_output["Result_FinancialStatement TotalAssets"] = stock_fins["TotalAssets"]
# stock_fins_output["Result_FinancialStatement NetAssets"],  純資産. JquantsAPIでは今とれなさそう
#stock_fins_output["Result_FinancialStatement NetSales"] = stock_fins["NetSales"]
#stock_fins_output["Result_FinancialStatement OperatingIncome"] = stock_fins["OperatingProfit"]
#stock_fins_output["Result_FinancialStatement OrdinaryIncome"] = stock_fins["OrdinaryProfit"]  # 経常利益
#stock_fins_output["Result_FinancialStatement NetIncome"] = stock_fins["Profit"]  # 当期純利益

#stock_fins_output